## Add, Modify and Retrieve data from OrientDB.


## 1.Install the pyorient package!

In [22]:
######## download pyorientdb 
! pip install pyorient --user 

## 2.Import packages and libraries.

In [23]:
import pyorient, json, pandas as pd
import sys

## 3. Add your service credentials for Object Storage
* You must create Object Storage service on Bluemix. To access data in a file in Object Storage, you need the Object Storage authentication credentials. Click on insert to code for the file you want to use in the code.


In [3]:
# # @hidden_cell
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_3fd48083_3bbd_4355_a296_f2558dc1b897',
  'project_id':'07704599ed444df0927832e455d9ec63',
  'region':'dallas',
  'user_id':'fb16c3d0955c42538faa723269123f5d',
  'domain_id':'1bfd0aa070764f39845124cf77d75a19',
  'domain_name':'1104675',
  'username':'member_b6c199f708035093454e23f2785a418ef096423b',
  'password':"""VO~S.?)5E6g2!!Dh""",
  'container':'graphdb',
  'tenantId':'undefined',
  'filename':'config.json'
}

from io import BytesIO  
import requests  
import json  
import pandas as pd

# loading the config.json

def get_data(credentials):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return json.loads(resp2.content)



node_data = get_data(credentials_1)
print node_data

{u'username': u'root', u'edge_class': {u'acted_in': {u'out': {u'Linked_Class': u'person', u'Type': u'Link'}, u'in': {u'Linked_Class': u'movie', u'Type': u'Link'}}, u'worked_with': {u'out': {u'Linked_Class': u'person', u'Type': u'Link'}, u'in': {u'Linked_Class': u'person', u'Type': u'Link'}}}, u'Database_name': u'testdb', u'vertex_class': {u'person': {u'role': u'String', u'name': u'String', u'fblikes': u'Float'}, u'movie': {u'title': u'String', u'numCriticForReviews': u'Integer', u'imdbRating': u'Float', u'durationInMins': u'Integer', u'year': u'Integer', u'genre': u'String', u'movieFacebookLikes': u'Float', u'plotKeywords': u'String'}}, u'host': u'localhost', u'password': u'root', u'port': 2424}


In [4]:
# loading the imdb movie data

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_3fd480833bbd4355a296f2558dc1b897(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_b6c199f708035093454e23f2785a418ef096423b','domain': {'id': '1bfd0aa070764f39845124cf77d75a19'},
            'password': 'VO~S.?)5E6g2!!Dh'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)


imdb_df= pd.read_csv(get_object_storage_file_with_credentials_3fd480833bbd4355a296f2558dc1b897('graphdb', 'INSIGHTS-GRAPHDB.csv'))

'''There are many rows and columns in the data that are empty. Hence, It is important to clean the data.
All the empty rows and columns are dropped from the dataframe using dropna() function of pandas.'''

imdb_df = imdb_df.dropna()
imdb_df.head(10)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,...,3054,English,USA,PG-13,237000000,2009,936,7.9,1.78,33000
1,Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,...,1238,English,USA,PG-13,300000000,2007,5000,7.1,2.35,0
2,Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,200074175,Action|Adventure|Thriller,...,994,English,UK,PG-13,245000000,2015,393,6.8,2.35,85000
3,Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,448130642,Action|Thriller,...,2701,English,USA,PG-13,250000000,2012,23000,8.5,2.35,164000
5,Color,Andrew Stanton,462,132,475,530,Samantha Morton,640,73058679,Action|Adventure|Sci-Fi,...,738,English,USA,PG-13,263700000,2012,632,6.6,2.35,24000
6,Color,Sam Raimi,392,156,0,4000,James Franco,24000,336530303,Action|Adventure|Romance,...,1902,English,USA,PG-13,258000000,2007,11000,6.2,2.35,0
7,Color,Nathan Greno,324,100,15,284,Donna Murphy,799,200807262,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,387,English,USA,PG,260000000,2010,553,7.8,1.85,29000
8,Color,Joss Whedon,635,141,0,19000,Robert Downey Jr.,26000,458991599,Action|Adventure|Sci-Fi,...,1117,English,USA,PG-13,250000000,2015,21000,7.5,2.35,118000
9,Color,David Yates,375,153,282,10000,Daniel Radcliffe,25000,301956980,Adventure|Family|Fantasy|Mystery,...,973,English,UK,PG,250000000,2009,11000,7.5,2.35,10000
10,Color,Zack Snyder,673,183,0,2000,Lauren Cohan,15000,330249062,Action|Adventure|Sci-Fi,...,3018,English,USA,PG-13,250000000,2016,4000,6.9,2.35,197000


##  4. Connecting to Orientdb


In [28]:
'''client = pyorient.OrientDB(<ip-address-of-the-kubernetes-cluster>,<node-port mapped to port 2424 of orientdb>)'''
print(client)

'''session-id username and password is global paswword, you have set for orientdb'''
session_id = client.connect(node_data['username'], node_data['password'])





## 5. Utility functions.



In [5]:
# method to check if class is present or not 
def check_if_class_present_or_not(classname):
    name="will be replaced by query result name"
    query = "SELECT FROM ( SELECT expand( classes ) FROM metadata:schema ) WHERE name = " +'"'+ classname + '"'
    print(query)
    a = client.command(query)
    
    for k in a:
        name = k.name
    
    if(name == classname):
        return True
    else:
        return False
    
# method to check if a person is present or not     
def check_if_already_present():
    check_if_already_present = "select * from person"
    c = client.command(check_if_already_present)
    d=[]

    for name in c:
        d.append([name.name, name.role, name.fblikes])


    check_df = pd.DataFrame(list(d), columns=['name','role','fblikes'])
    return check_df


# method to check if a movie is present or not 
def check_if_already_present_movie():
    check_if_already_present = "select title , year from movie"
    c = client.command(check_if_already_present)
  
    d=[]

    for name in c:
        d.append([name.title, name.year])


    check_df_movie = pd.DataFrame(list(d), columns=['title','year'])
    return check_df_movie

## method to find the cluster of a class 
def find_the_Cluster_id_of_a_class (classname):
    find_the_Cluster_id_of_a_class = "SELECT defaultClusterId from (SELECT expand( classes ) FROM metadata:schema) where name = '" + classname + "'"
    print(find_the_Cluster_id_of_a_class)
    c = client.command(find_the_Cluster_id_of_a_class)
    for ids in c:
        return ids.defaultClusterId




## 6. Core Functions


In [11]:
#  Method for creating Database
def createDatabase(node_data):
    # check if it exists or not
    if client.db_exists( node_data['Database_name'], pyorient.STORAGE_TYPE_MEMORY ):
        client.db_open(node_data['Database_name'],node_data['username'], node_data['password'])
        print node_data['Database_name'] + " "+ "has already been created and opened for use"
    else:
    # create the database
        client.db_create( node_data['Database_name'], pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY )
        print node_data['Database_name'] + " created and opened successfully"
        

# Method for createNodeClass_withSchema
def createNodeClass_withSchema(node_data):
    for class_name,value in node_data['vertex_class'].items():
        bool_result = check_if_class_present_or_not(class_name)
        print bool_result
        if(not bool_result):
            command_to_create_node = "create class"+" "+ class_name +" "+ "extends V"
            print(command_to_create_node)
            cluster_id = client.command(command_to_create_node) 
            print class_name,cluster_id
            for property_name,value in node_data['vertex_class'][class_name].items():
                ### create properties 
                command_to_create_property= "create property"+ " "+ class_name +"." + property_name +" " +"IF NOT EXISTS " + value
                print(command_to_create_property)
                client.command(command_to_create_property) 
            print "class " + class_name + " and its properties successfully created"
        else:
            print "This class has been created already."

# method for creating createNodeClass_NoSchema.
def createNodeClass_NoSchema(node_data):
    for class_name,value in node_data['vertex_class'].items():
        if(check_if_class_present_or_not(class_name)):
            command_to_create_node = "create class"+" "+ class_name +" "+ "extends V"
            print(command_to_create_node)
            cluster_id = client.command(command_to_create_node) 
            print class_name + " has been created with cluster id " +cluster_id
        else:
            print "This class has been created already."
            
# method for creating edge class.
def createEdgeClass(node_data):
    for class_name,v in node_data['edge_class'].items():
        if(not check_if_class_present_or_not(class_name)):
            command_to_create_edge_class = "create class"+" "+ class_name +" "+ "extends E"
            print(command_to_create_edge_class)
            cluster_id = client.command(command_to_create_edge_class)
            print("Edge class" +" " + class_name + "has been created successfully!")

            for key,val in node_data['edge_class'][class_name].items():
                print key,val
                command_to_create_property= "create property"+ " "+ class_name +"." + key +" " +"IF NOT EXISTS " + node_data['edge_class'][class_name][key]['Type']+" " + node_data['edge_class'][class_name][key]['Linked_Class']
                print(command_to_create_property)
                client.command(command_to_create_property) 
                print node_data['edge_class'][class_name][key]['Type'], node_data['edge_class'][class_name][key]['Linked_Class']
        else:
             print "This class has been created already."
            


# method to create nodes with their attributes.
def creating_records(imdb_df):
    for index, row in imdb_df.iterrows():
        ''' creating records -- all three actors and director 
        '''
        check_df = check_if_already_present()
        if(any(check_df.name == row["actor_1_name"])):
            print "node "+row["actor_1_name"] +" is already present!!"
        else:
            command_to_create_actor_1_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_1_name"]+'"' + ','+ str(row["actor_1_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_1_node_class_person)
            print "node "+row["actor_1_name"] +" has been created!!"
            
        if(any(check_df.name == row["actor_2_name"])):
            print "node "+row["actor_2_name"] +" is already present!!"
        else:
            command_to_create_actor_2_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_2_name"]+'"' + ','+ str(row["actor_2_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_2_node_class_person)
            print "node "+row["actor_2_name"] +" has been created!!"
            
        if(any(check_df.name == row["actor_3_name"])):
            print "node "+row["actor_3_name"] +" is already present!!"
        else:
            command_to_create_actor_3_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_3_name"]+'"' + ','+ str(row["actor_3_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_3_node_class_person)
            print "node "+row["actor_3_name"] +" has been created!!"
            
        if(any(check_df.name == row["director_name"])):
            print "node "+row["director_name"] +" is already present!!"
        else:
            command_to_create_director_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["director_name"]+'"' + ','+ str(row["director_facebook_likes"])+',' +'"' +'director'+'"' + ")"
            client.command(command_to_create_director_node_class_person)
            print "node "+row["director_name"] +" has been created!!"
        
        command_to_create_movie  = "INSERT INTO movie (title, year, durationInMins, imdbRating, genre, plotKeywords, numCriticForReviews, movieFacebookLikes) VALUES (" +'"' +row["movie_title"]+'"' + ','+ str(row["title_year"])+','+str(row["duration"])+','+str(row["imdb_score"])+',' +'"'+row["genres"]+'"' +','+'"' +row["plot_keywords"]+'"'+','+str(row["num_critic_for_reviews"])+','+str(row["movie_facebook_likes"])+")"
        print command_to_create_movie
        client.command(command_to_create_movie)
        print "node "+row["movie_title"] +" has been created!!"
        
        

        
# method to create the edge when you have designed schema for your graphdb       
def createRelationships():
      for index, row in imdb_df.iterrows():
            '''create_edge_between_two_actors_1 and actor_2 
            '''
            if((row["actor_1_name"]  in check_df.name.values) and (row["actor_2_name"]  in check_df.name.values)):
                command_to_create_edge_between_two_actors_1_and_2 = "create edge worked_with from (select from person where name = "+'"'+row["actor_1_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_2_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_1_and_2)
                client.command(command_to_create_edge_between_two_actors_1_and_2)
                print "edge has been created successfully!"
            else:
                   print "edge cant be created because vertext is not present !!!!!"
            
            
            '''create_edge_between_two_actors_1 and actor_2 
            '''
            if((row["actor_2_name"]  in check_df.name.values) and  (row["actor_3_name"]  in check_df.name.values) ):
                command_to_create_edge_between_two_actors_2_and_3 = "create edge worked_with from (select from person where name = "+'"'+row["actor_2_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_3_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_2_and_3)
                client.command(command_to_create_edge_between_two_actors_2_and_3)
                print "edge has been created successfully!"
            else:
                   print "edge cant be created because one or both the vertices is not present !!!!!"
               

            '''create_edge_between_two_actors_1 and actor_2
            '''
            if((row["actor_2_name"]  in check_df.name.values) and ( row["actor_3_name"]  in check_df.name.values )):
                command_to_create_edge_between_two_actors_3_and_1 = "create edge worked_with from (select from person where name = "+'"'+row["actor_3_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_1_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_3_and_1)
                client.command(command_to_create_edge_between_two_actors_3_and_1)
                print "edge has been created successfully!"
            else:
                   print "edge cant be created because one or both the vertices  is not present !!!!!"
                
            '''create_edge_between actor_1 and director 
            '''
            if((row["actor_1_name"]  in check_df.name.values) and  (row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_1_and_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_1_name"]+'"'+")"
                print(command_to_create_edge_between_actor_1_and_director)
                client.command(command_to_create_edge_between_actor_1_and_director)
            else:
                   print "edge cant be created because one or both the vertices is not present !!!!!"

            '''create_edge_between actor_2 and director
            '''
            if((row["actor_2_name"]  in check_df.name.values) and ( row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_2_and_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_2_name"]+'"'+")"
                print(command_to_create_edge_between_actor_2_and_director)
                client.command(command_to_create_edge_between_actor_2_and_director)
            else:
                   print "edge cant be created because one or both the vertices  is not present !!!!!"
            
            '''create_edge_between actor_3 and director 
            '''
            if((row["actor_3_name"]  in check_df.name.values) and ( row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_3_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_3_name"]+'"'+")"
                print(command_to_create_edge_between_actor_3_director)
                client.command(command_to_create_edge_between_actor_3_director)
            else:
                   print "edge cant be created because one or both the vertices is is not present !!!!!"

            '''movies and actors 
            '''
            '''create_edge_between_actors_1_and_movie
            '''
            if((row["actor_1_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values)):
                command_to_create_edge_between_actors_1_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_1_name"]+'"'+ ")"+" " +"to (select from movie where title = "+'"'+row["movie_title"]+'"'+")"
                print(command_to_create_edge_between_actors_1_and_movie)
                client.command(command_to_create_edge_between_actors_1_and_movie)
            else:
                   print "edge cant be created because one or both the vertices is not present !!!!!"

            '''create_edge_between_actors_2_and_movie
            '''
            if((row["actor_2_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values) ):
                command_to_create_edge_between_actors_2_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_2_name"]+'"'+ ")"+" " +"to (select from movie where title = "+'"'+row["movie_title"]+'"'+")"
                print( command_to_create_edge_between_actors_2_and_movie )
                client.command(command_to_create_edge_between_actors_2_and_movie)
            else:
                   print "edge cant be created because one or both the vertices is not present !!!!!"
            
            '''create_edge_between_actors_3_and_movie 
            '''
            if((row["actor_3_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values)):
                command_to_create_edge_between_actors_3_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_3_name"]+'"'+ ")"+" " +"to (select from movie where title= "+'"'+row["movie_title"]+'"'+")"
                print(command_to_create_edge_between_actors_3_and_movie)
                client.command(command_to_create_edge_between_actors_3_and_movie)
            else:
                   print "edge cant be created because one or both the vertices is not present !!!!!"

        

# Method to insert/create the records for the usecase where there is no defined schema.
def creating_records_noschema(data):
        id = client.record_create(cluster_id, data)
        print "record succesfully created with " + str(id)
        

# Most Mentioned: Movie with maximum no of facebook likes. 
def most_mentioned_movie():
    a = client.command(('select max(movieFacebookLikes) from movie '))

    for max_num in a :
        max_num = max_num.max
    print max_num
    most_mentioned_movie_object = client.command('select title from movie where movieFacebookLikes = ' + str(max_num))
    
    for titles in most_mentioned_movie_object :
        print titles.title
        

# Clustering:Cluster movies with rating above 7.
def movie_with_imdb_rating_above_7():
    title = []
    a = client.command(('select title from movie where imdbRating > 7 '))
    for titles in a :
        title.append(titles.title)
    title_df = pd.DataFrame(list(title), columns=['title'])    
    return title_df
        

     

In [ ]:
createDatabase(node_data)
createNodeClass_withSchema(node_data)
createEdgeClass(node_data)
creating_records(imdb_df)
check_df = check_if_already_present()
check_df_movie = check_if_already_present_movie()
createRelationships()


## 7. Inserting/Creating the records for the usecase where there is ` no defined schema.`

In [ ]:
data = {
        'person': {
                "name": "John", 
                "role": "director",
                "fblikes": 400000.0,
                "born_in" : 1980
                },
        }

for key, value in data.iteritems():
        cluster_id = find_the_Cluster_id_of_a_class(key)
        print cluster_id


creating_records_noschema(data)


SELECT defaultClusterId from (SELECT expand( classes ) FROM metadata:schema) where name = 'person'


## 8. Insights from orientDB/querying the Orientdb

In [174]:
most_mentioned_movie = most_mentioned_movie()
print most_mentioned_movie
movie_with_imdb_rating_above_7 = movie_with_imdb_rating_above_7()
print movie_with_imdb_rating_above_7

349000
InterstellarÂ
